In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Initialize list to store all locations
locations = []

# Find all state sections
state_sections = soup.find_all('div', class_='subregions_page_locations_container')[0].find_all('div', recursive=False)

for section in state_sections:
    # Check if this section has a state header
    state_header = section.find('h4', class_='subregion_header')
    if not state_header:
        continue
        
    state = state_header.text.strip()
    
    # Find all locations in this state
    location_containers = section.find_all('div', class_='subregion_location')
    
    for container in location_containers:
        # Extract location name
        title_elem = container.find('div', class_='subregion_location_title')
        location_name = title_elem.find('p').text.strip() if title_elem and title_elem.find('p') else "N/A"
        
        # Extract address lines
        address_lines = []
        address_elems = container.find_all('div', class_='subregion_location_addressLine1')
        for elem in address_elems:
            if 'subregion_location_title' not in elem.get('class', []):  # Avoid double-counting title
                address_lines.append(elem.text.strip())
        full_address = ", ".join(address_lines)
        
        # Extract phone number
        phone_elem = container.find('a', href=re.compile(r'tel:'))
        phone = phone_elem.text.strip() if phone_elem else "N/A"
        
        # Extract coordinates from demo drive link
        demo_link = container.find('a', href=re.compile(r'/drive\?'))
        lat, lng = "N/A", "N/A"
        if demo_link and 'href' in demo_link.attrs:
            href = demo_link['href']
            lat_match = re.search(r'lat=([-0-9.]+)', href)
            lng_match = re.search(r'lng=([-0-9.]+)', href)
            lat = lat_match.group(1) if lat_match else "N/A"
            lng = lng_match.group(1) if lng_match else "N/A"
        
        # Add to locations list
        locations.append({
            'State': state,
            'Location Name': location_name,
            'Full Address': full_address,
            'Phone Number': phone,
            'Latitude': lat,
            'Longitude': lng
        })

# Create DataFrame
df = pd.DataFrame(locations)

# Save to Excel
df.to_excel('tesla_locations.xlsx', index=False)

print(f"Extracted {len(locations)} locations. Saved to tesla_locations.xlsx")

RuntimeError: Could not fetch the Tesla U.S. Stores page (received repeated 403/blocked responses). Try again later or switch to a headless browser renderer (Selenium/Playwright).